In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
num_shots = 3

In [3]:
import pandas as pd

In [4]:
experiment_name = "2024.05.09-02_Llama3_8B-Instruct_bfloat-16_3-few-shot"

In [5]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

#top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes_with_2023.12.11_input_data.pickle"

output_data_path = base_path + "/../Llama-3/data/output/" + experiment_name
log_filename = base_path + "/../Llama-3/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [6]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) #|
        # (data_sel_4_few_shot_examples.text_id==370)  |
        # (data_sel_4_few_shot_examples.text_id==1422) |
        # (data_sel_4_few_shot_examples.text_id==4329) |
        # (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(3)

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125,47
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000,136


In [34]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    # output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    output_text = "Output: \"SDG-"+str(row['sdg'])+'\"\n\n'

    few_shots += input_text
    few_shots += output_text

print(few_shots)

Input text: "As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far."
Output: "SDG-8"

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Output: "SDG-12"

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 

In [8]:
from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [9]:
!huggingface-cli login --token $huggingface_cli_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
!pip install -q accelerate bitsandbytes rich transformers --progress-bar off

In [11]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig

In [12]:
import pickle

In [13]:
# from IPython.display import clear_output
# huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
# clear_output()

In [14]:
# !huggingface-cli login --token $huggingface_cli_token

In [15]:
# pretrained_model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# model = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path=pretrained_model_name_or_path,
#     quantization_config=quantization_config
# )


# tokenizer = AutoTokenizer.from_pretrained(
#     pretrained_model_name_or_path=pretrained_model_name_or_path
# )

In [16]:
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

pretrained_model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = pretrained_model_name_or_path

# pipe = pipeline(
#     "text-generation",
#     model=pretrained_model_name_or_path,
#     model_kwargs={"torch_dtype": torch.bfloat16, "bb_config": quantization_config},
#     device_map="auto",
# )

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [18]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

## Run the model

In [19]:
data_sel = data #.iloc[:3]

In [20]:
# def generate_query_prompt(query: str) -> str:
#   """Generate LLM prompt for user query."""
#   prompt = (
#       "<|system|>\n"
#       "You are a text classifier.</s>\n"
#       "<|user|>\n"
#       f"{query}.</s>\n"
#       "<|assistant|>\n"
#   )
#   return prompt

In [35]:
text_id = data_sel.iloc[0].text_id
text = data_sel.iloc[0].text.replace('\n','').replace('{','(').replace('}',')')
sdg_true = data_sel.iloc[0].sdg

print("EXPECTED_SDG:", sdg_true, "\nTEXT:", text.replace('\n',''),'\n\n')

template = f"""Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

"SDG-1": "End poverty in all its forms everywhere."
"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
"SDG-5": "Achieve gender equality and empower all women and girls."
"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
"SDG-10": "Reduce inequality within and among countries."
"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
"SDG-12": "Ensure sustainable consumption and production patterns."
"SDG-13": "Take urgent action to combat climate change and its impacts."
"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
"SDG-0": "Other."

Choose ONLY ONE label for each input text.

DO NOT include the input text in your answer.

The following are """+str(num_shots)+f""" examples of expected classification given an input text:

"""+few_shots+f"""

The input text is:
'''
{text}
'''
"""

# prompt = generate_query_prompt(query=template)
# print(prompt)
template

EXPECTED_SDG: 0 
TEXT: L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News. 




'Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:\n\n"SDG-1": "End poverty in all its forms everywhere."\n"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."\n"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."\n"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."\n"SDG-5": "Achieve gender equality and empower all women and girls."\n"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."\n"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."\n"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."\n"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."\n"SDG-

In [22]:
messages = [
    {"role": "system", "content": "You are a text classifier."},
    {"role": "user", "content": template},
]

In [23]:
# prompt = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
# )

In [24]:
# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

In [25]:
# output = pipe(text_inputs=template, max_new_tokens=1024, temperature=0.1, return_full_text=False)
# output



In [26]:
# outputs = pipeline(
#     prompt,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.1,
#     top_p=0.9,
# )

In [27]:
# print(outputs[0]["generated_text"][len(prompt):])

In [28]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)


In [29]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [30]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [31]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

SDG-11


In [ ]:
# results = []

# for index, row in data_sel.iterrows():

#   cnt = index

#   text_id = row['text_id']
#   text = row["text"].replace('\n','').replace('{','(').replace('}',')')
#   sdg_true = row["sdg"]

#   file = open(log_filename, 'a+', encoding='utf-8')

#   print("\n\n##################################################")
#   file.write("\n\n##################################################\n")

#   print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')
#   file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + row['text'].replace('\n','') + "\n\n\n")

#   template = f"""Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

# "SDG-1": "End poverty in all its forms everywhere."
# "SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
# "SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
# "SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
# "SDG-5": "Achieve gender equality and empower all women and girls."
# "SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
# "SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
# "SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
# "SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
# "SDG-10": "Reduce inequality within and among countries."
# "SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
# "SDG-12": "Ensure sustainable consumption and production patterns."
# "SDG-13": "Take urgent action to combat climate change and its impacts."
# "SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
# "SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
# "SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
# "SDG-0": "Other."

# Choose ONLY ONE label for each input text.

# DO NOT include the input text in your answer.

# The following are """+str(num_shots)+f""" examples of expected classification given an input text:

# """+few_shots+f"""

# The input text is:
# '''
# {text}
# '''
# """

#   messages = [
#       {"role": "system", "content": "You are a text classifier."},
#       {"role": "user", "content": template},
#   ]

#   input_ids = tokenizer.apply_chat_template(
#       messages,
#       add_generation_prompt=True,
#       return_tensors="pt"
#   ).to(model.device)

#   terminators = [
#       tokenizer.eos_token_id,
#       tokenizer.convert_tokens_to_ids("<|eot_id|>")
#   ]

#   outputs = model.generate(
#       input_ids,
#       max_new_tokens=256,
#       eos_token_id=terminators,
#       do_sample=True,
#       temperature=0.6,
#       top_p=0.9,
#       pad_token_id=tokenizer.eos_token_id
#   )

#   response = outputs[0][input_ids.shape[-1]:]

#   output = tokenizer.decode(response, skip_special_tokens=True)


#   result_extended = (cnt, text_id, sdg_true, output)

#   print(result_extended)
#   file.write(str(result_extended)+'\n')

#   with open(output_data_path+f'/sdg_classification_{cnt}.pkl', 'wb') as f:
#       pickle.dump(result_extended, f)

#   file.close()


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




##################################################
EXPECTED_SDG: 0 TEXT: L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News. 




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


(0, 6668, 0, '---SDG 0---')


##################################################
EXPECTED_SDG: 0 TEXT: Brooklyn Decker’s SI Swimsuit Cover Photo Shoot in the Maldives Is Beyond Beautiful  Yahoo Life. 




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

(1016, 6628, 16, 'SDG-4')


##################################################
EXPECTED_SDG: 16 TEXT: Accountability of government officials' actions is strongly related to the exercise of government authority. In carrying out duties to realize the general welfare, the authority used by organs or government officials is based on the provisions of the laws and regulations (the principle of legality). However, it is not uncommon for the task to be carried out based on discretionary authority. The freedom of government officials to make decisions based on discretionary authority has a great potential to be abused which results in consequences from both point of view of administrative law as well as of criminal law. In the practice, there is discrepancy among law enforcers on the understanding of the principles related to the accountability of discretionary authority held by government officials. 




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


(1017, 4563, 16, '---SDG 16---')


##################################################
EXPECTED_SDG: 16 TEXT: In Phillips Petroleum Co. v. Shutts, the United States Supreme Court held that plaintiff class members residing outside the forum state are entitled to the "minimal due process protections" of notice, an opportunity to be heard, an opportunity to opt out, and adequate representation. However, class actions involving class members not just from other states but from other countries raise distinctive due process concerns in each of these areas. In this Article, Professor Bassett examines the considerations impacting on due process and personal jurisdiction when non-U.S. claimants participate in class litigation, and proposes guidelines necessary to ensure that the class judgment will have a binding effect on foreign claimants. 




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


(1018, 6710, 16, '---SDG 16---')


##################################################
EXPECTED_SDG: 16 TEXT: For the past half a century, Latin American scholars have been pointing toward the emergence of new social actors as agents of social and political democratization. The first wave of actors was characterized by the emergence of novel agents—mainly, new popular movements—of social transformation. At first, the second wave, epitomized by nongovernmental organizations (NGOs), was celebrated as the upsurge of a new civil society, but later on, it was the target of harsh criticism. The literature often portrays this development in Latin American civil society as a displacement trend of actors of the first wave by the second wave—“NGOization”—and even denounces new civil society as rootless, depoliticized, and functional to retrenchment. Thus, supposedly, NGOization encumbers social change. The authors argue that NGOization diagnosis is a flawed depiction of change within civil societ

In [ ]:
results = []

for index, row in data_sel.iterrows():

  cnt = index

  text_id = row['text_id']
  text = row["text"].replace('\n','').replace('{','(').replace('}',')')
  sdg_true = row["sdg"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')
  file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  template = f"""Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

"SDG-1": "End poverty in all its forms everywhere."
"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
"SDG-5": "Achieve gender equality and empower all women and girls."
"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
"SDG-10": "Reduce inequality within and among countries."
"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
"SDG-12": "Ensure sustainable consumption and production patterns."
"SDG-13": "Take urgent action to combat climate change and its impacts."
"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
"SDG-0": "Other."

Choose ONLY ONE label for each input text.

DO NOT include the input text in your answer.

The following are """+str(num_shots)+f""" examples of expected classification given an input text:

"""+few_shots+f"""

The input text is:
'''
{text}
'''
"""

  messages = [
      {"role": "system", "content": "You are a text classifier."},
      {"role": "user", "content": template},
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
      pad_token_id=tokenizer.eos_token_id
  )

  #model.generate(**encoded_input, pad_token_id=tokenizer.eos_token_id)

  response = outputs[0][input_ids.shape[-1]:]

  output = tokenizer.decode(response, skip_special_tokens=True)


  result_extended = (cnt, text_id, sdg_true, output)

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  file.close()




##################################################
EXPECTED_SDG: 0 TEXT: L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News. 


(0, 6668, 0, 'SDG-0')


##################################################
EXPECTED_SDG: 0 TEXT: Brooklyn Decker’s SI Swimsuit Cover Photo Shoot in the Maldives Is Beyond Beautiful  Yahoo Life. 


(1, 6099, 0, 'SDG-0')


##################################################
EXPECTED_SDG: 0 TEXT: Mumbai: The Bombay High Court has suspended the life sentence awarded to Afroz Khan Pathan, a convict in the sensational 2006 Aurangabad arms haul case, and granted him conditional bail . A division bench comprising Justice Revati Mohite-Dere and Justice Gauri Godse also observed that there was no prima facie evidence to prove that Pathan had visited Bangladesh

In [ ]:
# !ls "/content/gdrive/MyDrive/Colab/SDG/data/../Llama-3/data/output/2024.05.09-02_Llama3_8B-Instruct_bfloat-16_3-few-shot"

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

1020

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

,cnt,text_id,sdg_true,result
0,21,3889,0,---SDG 0---
1,22,6576,0,---SDG 0---
2,23,3912,0,---SDG 11---


In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
# results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
# results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
# results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
# results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')

results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
tmp_df_1 = results_processed[~results_processed.label.isna()]
# tmp_df_1

results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
tmp_df_2 = results_processed[~results_processed.label.isna()]
# tmp_df_2
# tmp_df_1 = results_processed['result'].str.extract(r'SDG-(\d+)')

results_processed = pd.concat([tmp_df_1,tmp_df_2])
results_processed

# results_processed['result'].str.extract(r'SDG-(\d+)')[~results_processed.label.isna()]

# results_processed.head(3)

NameError: name 'results_processed' is not defined

In [ ]:
# results_processed[results_processed.text_id==6144]

In [ ]:
# results_processed[results_processed.label.isna()] #.str.extract(r'---SDG (\d+)---')

In [ ]:
# results_processed[results_processed.label.isna()]

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,---SDG 0---,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,---SDG 5---,5
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,---SDG 16---,16


In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

[]

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.67      0.37      0.47        60
           1       0.70      0.62      0.65        60
           2       0.88      0.47      0.61        60
           3       0.83      0.63      0.72        60
           4       0.36      1.00      0.52        60
           5       0.72      0.88      0.79        60
           6       0.88      0.58      0.70        60
           7       0.79      0.87      0.83        60
           8       0.22      0.43      0.29        60
           9       0.35      0.32      0.33        60
          10       0.70      0.43      0.54        60
          11       0.82      0.38      0.52        60
          12       0.64      0.82      0.72        60
          13       0.71      0.68      0.69        60
          14       0.96      0.40      0.56        60
          15       0.88      0.70      0.78        60
          16       0.65      0.68      0.67        60

    accuracy    

In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [ ]:
results_processed_merged = final

In [ ]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['label']) or row['label'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'label'] = 0  # new_value

In [ ]:
results_processed_merged.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,---SDG 0---,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,---SDG 5---,5
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,---SDG 16---,16


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.67      0.37      0.47        60
           1       0.70      0.62      0.65        60
           2       0.88      0.47      0.61        60
           3       0.83      0.63      0.72        60
           4       0.36      1.00      0.52        60
           5       0.72      0.88      0.79        60
           6       0.88      0.58      0.70        60
           7       0.79      0.87      0.83        60
           8       0.22      0.43      0.29        60
           9       0.35      0.32      0.33        60
          10       0.70      0.43      0.54        60
          11       0.82      0.38      0.52        60
          12       0.64      0.82      0.72        60
          13       0.71      0.68      0.69        60
          14       0.96      0.40      0.56        60
          15       0.88      0.70      0.78        60
          16       0.65      0.68      0.67        60

    accuracy    

In [ ]:
output_datapath = output_data_path

In [ ]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [ ]:
report_df

,precision,recall,f1-score,support
0,66.666667,36.666667,47.311828,60.000000
1,69.811321,61.666667,65.486726,60.000000
2,87.500000,46.666667,60.869565,60.000000
3,82.608696,63.333333,71.698113,60.000000
4,35.502959,100.000000,52.401747,60.000000
5,71.621622,88.333333,79.104478,60.000000
6,87.500000,58.333333,70.000000,60.000000
7,78.787879,86.666667,82.539683,60.000000
8,22.222222,43.333333,29.378531,60.000000
9,34.545455,31.666667,33.043478,60.000000


In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

68.9954500415323 60.392156862745104 60.3921568627451 61.18708850498593


In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

69.0 60.4 60.4 61.2
